In [2]:
!pip install pymupdf

164.62s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [4]:
!pip install pymongo

406.78s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.7/748.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pymongo]m1/2 [pymongo]


In [8]:
import fitz  # PyMuPDF
import os
import json
from pprint import pprint

In [9]:
DATA_DIR = "./PDFs"

pdf_files = [f for f in os.listdir(DATA_DIR) if f.endswith(".pdf")]
print(f"Found {len(pdf_files)} PDF(s):")
pprint(pdf_files)

Found 2 PDF(s):
['metformin2.pdf', 'metformin1.pdf']


In [10]:
def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    pages = []
    for page_number in range(len(doc)):
        page = doc.load_page(page_number)
        text = page.get_text().strip()
        pages.append({
            "page_number": page_number + 1,
            "text": text
        })
    doc.close()
    return pages

In [11]:
# Dictionary to hold page-wise text for each PDF
pdf_page_texts = {}

for pdf_file in pdf_files:
    full_path = os.path.join(DATA_DIR, pdf_file)
    print(f"🔍 Extracting: {pdf_file}")
    pages = extract_text_from_pdf(full_path)
    pdf_page_texts[pdf_file] = pages

print("✅ Extraction complete.")

🔍 Extracting: metformin2.pdf
🔍 Extracting: metformin1.pdf
✅ Extraction complete.


In [20]:
# Show the first 2 pages of the first PDF
sample_pdf = pdf_files[0]
pprint(pdf_page_texts[sample_pdf][:1])

[{'page_number': 1,
  'text': 'PRODUCT  MONOGRAPH \n'
          ' \n'
          ' \n'
          ' \n'
          ' \n'
          'PrTEVA-METFORMIN \n'
          '(metformin hydrochloride) \n'
          ' \n'
          '500 mg and 850 mg Tablets \n'
          ' \n'
          'Teva Standard \n'
          ' \n'
          ' \n'
          'Oral Antihyperglycemic Agent \n'
          ' \n'
          ' \n'
          ' \n'
          ' \n'
          'Teva Canada Limited  \n'
          ' \n'
          ' \n'
          '                              Date of Preparation: July 25, 2013 \n'
          '30 Novopharm Court  \n'
          ' \n'
          ' \n'
          ' \n'
          ' \n'
          ' \n'
          'Toronto, Ontario \n'
          ' \n'
          'Canada M1B 2K9 \n'
          'www.tevacanada.com  \n'
          ' \n'
          'Control Number: 165821'}]


In [27]:
%run ./utils.ipynb

✅ Successfully connected to MongoDB.


In [28]:
client = get_mongo_client(uri)
collection = get_collection(client, "pdf_rag_db", "pages")

In [29]:
for pdf_name, pages in pdf_page_texts.items():
    print(f"📤 Inserting pages from: {pdf_name}")
    insert_pdf_pages_to_mongo(collection, pdf_name, pages)

print("✅ All PDFs inserted into MongoDB.")

📤 Inserting pages from: metformin2.pdf
✅ Inserted 31 pages from 'metformin2.pdf' into MongoDB.
📤 Inserting pages from: metformin1.pdf
✅ Inserted 35 pages from 'metformin1.pdf' into MongoDB.
✅ All PDFs inserted into MongoDB.
